In [ ]:
from serpapi import GoogleSearch
import requests
import time
import re

SERPAPI_KEY = "bd0d819e093e6c8acf32d6aaab551208bb2f15b199966fa215daa790ba73c33d"
GROQ_API_KEY = "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"
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"

def retry_with_backoff(func, *args, max_retries=5, base_delay=1, **kwargs):
    """
    Retry a function with exponential backoff on HTTP 429 errors.
    """
    retries = 0
    while retries < max_retries:
        try:
            return func(*args, **kwargs)
        except requests.exceptions.HTTPError as http_err:
            if http_err.response.status_code == 429:
                # Extract wait time from error message, e.g., "Please try again in 6.363s"
                error_message = http_err.response.json().get("error", {}).get("message", "")
                wait_time_match = re.search(r"Please try again in ([\d.]+)s", error_message)
                wait_time = float(wait_time_match.group(1)) if wait_time_match else base_delay * (2 ** retries)
                time.sleep(wait_time)
                retries += 1
            else:
                raise http_err
        except Exception as e:
            raise e
    print(f"⚠️ Failed to process request for {args[0]} after {max_retries} retries due to rate limits.")
    return ""

def get_places(query):
    params = {
        "engine": "google_maps",
        "q": query,
        "api_key": SERPAPI_KEY
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    
    places = []
    try:
        for place in results.get("local_results", []):
            name = place.get("title", "Unknown")
            address = place.get("address", "Unknown")
            place_id = place.get("place_id", "")
            print(f"\n✅ Found place: {name} ({address})")
            if place_id:
                places.append((name, address, place_id))
    except Exception as e:
        print("❌ Could not get places:", e)
    return places

def get_reviews_by_place_id(place_id):
    params = {
        "engine": "google_maps_reviews",
        "place_id": place_id,
        "api_key": SERPAPI_KEY
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    try:
        reviews = results.get("reviews", [])
        print(f"\n📝 Found {len(reviews)} reviews")
        return reviews
    except Exception as e:
        print("❌ Could not fetch reviews:", e)
        return []

def generate_vibe_summary(location, reviews):
    if not reviews:
        print("❌ No reviews provided.")
        return ""

    review_snippets = "\n".join(
        [f"{i+1}. \"{r['snippet']}\" (Rating: {r['rating']})" for i, r in enumerate(reviews[:10])]
    )

    prompt = f"""
You are VibeBot, a fun and friendly AI that summarizes the vibe of places based on real Google Maps reviews.

Place: {location}

Here are some user reviews:
{review_snippets}

Now write a short and engaging summary of the vibe of this place:
- Use emojis and a playful tone.
- Add vibe tags like #cozy, #lively, #chill, etc.
- Include 2–3 short citations from users like "One reviewer said..."
"""

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "llama3-8b-8192",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.8
    }

    def make_request():
        response = requests.post(GROQ_URL, json=payload, headers=headers)
        response.raise_for_status()
        result = response.json()
        return result["choices"][0]["message"]["content"]

    try:
        summary = retry_with_backoff(make_request)
        if summary:
            print("\n🎯 Vibe Summary:\n")
            print(summary)
        return summary
    except Exception as e:
        print(f"⚠️ Could not generate summary for {location}.")
        return ""

if __name__ == "__main__":
    query = input("🔍 Enter location search (e.g., 'cafes in Bangalore'): ")
    places = get_places(query)

    for name, address, place_id in places:
        reviews = get_reviews_by_place_id(place_id)
        if reviews:
            print(f"\n🎉 Summary for {name}, {address}")
            generate_vibe_summary(f"{name}, {address}", reviews)
        else:
            print(f"⚠️ No reviews found for {name}, skipping.")